## Redo pangenome stuff with new sourmash pangenome plugin. 
- Clean up this repo
- try pangenome in protein space using translated sketches (use 10 v 10 now)
- need to deduplicate the genomes at 99% ANI, then run Roary
- Use roary, then sourmash

In [ ]:
# update plugin
mamba activate pangenomics_dev

# upgrade
pip install --upgrade --force-reinstall git+https://github.com/dib-lab/sourmash_plugin_pangenomics.git


## Steps + What do we want. 
I want to be able to compare roary output with the metagenomes. I want to do this in protein space, as roary works in protein space and that seems like the right comparison. 
I also want to do it in DNA space. I'd like to find a species, that has new strains in pig, but also a reasonable amount of reference strains. 

### Steps:
- Find a species
- Download all strains from genbank/gtdb
- dereplicate at 99% ANI
- run prokka
- Run roary
- Translate 10 metaGs
- make a cloud and a core sketch. 
- Run these against the metagenomes

Start with reference genomes, then add a pipeline for adding MAGs from pigs. 
translate metags

In [ ]:
# where is the tax file:
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250411_mag_taxonomy.tsv

In [ ]:
# Do a test with just reference genomes. (Megasphaera elsdenii)
mkdir test_pipeline

# see Snakefile
# how to go from 2 lists of species to a folder with all of them: For drep. 
srun --account=ctbrowngrp -p med2 -J roary -t 2:00:00 -c 10 --mem=50gb --pty bash
mamba activate branchwater-skipmer

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -s gather.smk -n

#### Going from a collection of dereplicated genomes to a pangenome sketch (different snakefile)

Want this in a separate snakefile because i cannot keep track of large snakefiles. 
This needs the following rules:
- prokka
- roary
- sourmash sketch (cloud and core), in protein and dna space

### Running the pangenome sketch against the metagenomes
- Download sra
- sketch in translate (metag is in wort in DNA)

In [ ]:
# gene presence absemce file
# how to get to those lists, then files?
python /group/ctbrowngrp2/amhorst/2025-pangenome/workflow/scripts/count_genes.py \
gene_presence_absence.Rtab test.csv

# gives files hard_core.txt, soft_core.txt, cloud.txt, shell.txt
# with group numbers. Now how to get the sequence?
# from pan_genome_reference.fa --> proteins?
# and easiest is probs converting these sequences to AA
mamba activate seqkit


seqkit grep -r -n -f /group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/roary/hard_core.txt \
/group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/roary/pan_genome_reference.fa \
-o /group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/sourmash/m_elsdenii.core.fa